In [1]:
import sys
sys.path.append("..") # for sibling import

import compyute as cp

Compyute: available devices ['cpu', 'cuda']


# Example 3.1

### Deep neural network using sequential model

The goal of this model is to classify iris species based on numerical features.

### Step 1: Prepare data
You will need to download the dataset from https://www.kaggle.com/datasets/uciml/iris and place it into the *data* directory.

In [2]:
# ! pip install pandas

In [3]:
import pandas as pd

data_orig = pd.read_csv('../data/iris.csv')
data = data_orig.copy()
data.drop(columns=['Id'], inplace=True)
data.head()

C:\Users\xboxd\AppData\Local\Temp\ipykernel_19208\2392560263.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa


In [4]:
data.describe()

,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm
count,150.000000,150.000000,150.000000,150.000000
mean,5.843333,3.054000,3.758667,1.198667
std,0.828066,0.433594,1.764420,0.763161
min,4.300000,2.000000,1.000000,0.100000
25%,5.100000,2.800000,1.600000,0.300000
50%,5.800000,3.000000,4.350000,1.300000
75%,6.400000,3.300000,5.100000,1.800000
max,7.900000,4.400000,6.900000,2.500000


In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   SepalLengthCm  150 non-null    float64
 1   SepalWidthCm   150 non-null    float64
 2   PetalLengthCm  150 non-null    float64
 3   PetalWidthCm   150 non-null    float64
 4   Species        150 non-null    object 
dtypes: float64(4), object(1)
memory usage: 6.0+ KB


The labels are categorical values. To be used in the model, all data needs to be numerical.

In [6]:
data_enc = data.copy()
data_enc["Species"] = data_enc["Species"].astype("category").cat.codes
data_enc.head()

,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0


Convert dataframe to a tensor.

In [7]:
data_tensor = cp.Tensor(data_enc.to_numpy())
data_tensor.shape

(150, 5)

Next the data is split into a training, validation and a testing dataset using the `split_train_test_val_data()` to evaluate the model later on. Before splitting the data is also shuffled, since sometimes raw data is sorted in some way.

In [8]:
from compyute.preprocessing import split_train_val_test

train, val, test = split_train_val_test(data_tensor, ratio_val=0.25, ratio_test=0.25)
train[:5]

Tensor([[4.9, 3.1, 1.5, 0.1, 0. ],
        [5.7, 3. , 4.2, 1.2, 1. ],
        [6.5, 3. , 5.8, 2.2, 2. ],
        [6.5, 3. , 5.5, 1.8, 2. ],
        [5. , 3.4, 1.6, 0.4, 0. ]], dtype='float64', shape=(5, 5), device='cpu')

Features and labels are now seperated.

In [9]:
X_train, y_train = train[:, :-1], train[:, -1].int()
X_val, y_val = val[:, :-1], val[:, -1].int()
X_test, y_test = test[:, :-1], test[:, -1].int()

In [10]:
y_train[:10]

Tensor([0, 1, 2, 2, 0, 1, 2, 0, 2, 2], dtype='int32', shape=(10,), device='cpu')

Neural networks tend to run into problems if values are very high. Therefore it is common to normalize the data. This can be done using the `normalize()` function, which applies min-max feature scaling to a tensor.<br><br>
$ X'=a+\frac{(X-X_{min})\cdot(b-a)}{X_{max}-X_{min}} $<br><br>, where<br><br>$ a $ ... lower bound<br>$ b $ ... upper bound

In [11]:
from compyute.preprocessing import normalize

X_train = normalize(X_train, axis=0, l_bound=-1)
X_val = normalize(X_val, axis=0, l_bound=-1)
X_test = normalize(X_test, axis=0, l_bound=-1)
X_train[:5]

Tensor([[-0.6667,  0.    , -0.8621, -1.    ],
        [-0.2222, -0.0909,  0.069 , -0.0833],
        [ 0.2222, -0.0909,  0.6207,  0.75  ],
        [ 0.2222, -0.0909,  0.5172,  0.4167],
        [-0.6111,  0.2727, -0.8276, -0.75  ]], dtype='float64', shape=(5, 4), device='cpu')

In [12]:
print (f'{X_train.shape=}')
print (f'{y_train.shape=}')

print (f'{X_val.shape=}')
print (f'{y_val.shape=}')

print (f'{X_test.shape=}')
print (f'{y_test.shape=}')

X_train.shape=(75, 4)
y_train.shape=(75,)
X_val.shape=(37, 4)
y_val.shape=(37,)
X_test.shape=(38, 4)
y_test.shape=(38,)


### Step 2: Build the neural network structure
Here the individual layers of the neural network models are defined.

In [13]:
from compyute import nn

model = nn.Sequential([
    nn.Linear(4, 16),
    nn.ReLU(),
    nn.Batchnorm1d(16),
    nn.Linear(16, 3),
])

The network is compiled to finalize the model. Besides the SGD optimizer, the framework also provides other algorithms like Adam. There are also multiple loss functions to choose from. Since this example explores a classification problem, the cross entropy loss function is used.

In [14]:
model.summary(input_shape=(4,))

Sequential
---------------------------------------------------------------
Layer                     Output Shape            # Parameters
Sequential                (-1, 3)                          163
 Linear                   (-1, 16)                          80
 ReLU                     (-1, 16)                           0
 Batchnorm1d              (-1, 16)                          32
 Linear                   (-1, 3)                           51

Total parameters: 163


### Step 3: Train the model

In [15]:
from compyute.nn.trainer import Trainer

trainer = Trainer(
    model=model,
    optimizer="sgd",
    loss="cross_entropy",
    metric="accuracy",
    verbose=1
)

In [16]:
trainer.train(X_train, y_train, epochs=300, val_data=(X_val, y_val))

In [17]:
# ! pip install matplotlib

In [18]:
import matplotlib.pyplot as plt

def plot_history(t1, t2):
    trace1 = trainer.state[t1]
    trace2 = trainer.state[t2]
    plt.figure(figsize=(10, 3))
    plt.plot(cp.arange(start=1, stop=len(trace1) + 1).to_numpy(), trace1, linewidth=1)
    plt.plot(cp.arange(start=1, stop=len(trace2) + 1).to_numpy(), trace2, linewidth=1)
    plt.legend([t1, t2])
    plt.grid(color="gray", linestyle="--", linewidth=0.5)

In [19]:
plot_history("loss", "val_loss")

KeyError: 'loss'

In [ ]:
plot_history("accuracy", "val_accuracy")

### Step 4: Evaluate the model
Using the defined metric, the model's performance can be evaluated using testing/validation data.

In [ ]:
loss, accuracy = trainer.evaluate_model(X_test, y_test)
print(f'loss {loss:.4f}')
print(f'accuracy {100*accuracy:.2f}')